# Gradio Demo For Facial Recognition on Missing Children Database

## Import Dependencies

In [1]:
#make sure to import these dependencies in this order
import os
import gradio as gr
from align import align_face
from PIL import Image
os.chdir('../')

In [2]:
#may or may not need to comment the following out 
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['NUMBAPRO_NVVM']='/share/pkg.7/cuda/11.2/install/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE']='/share/pkg.7/cuda/11.2/install/nvvm/libdevice/'
import segmentation_models as sm
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

Segmentation Models: using `tf.keras` framework.


In [3]:
import tensorflow as tf
import keras
import keras_vggface
from keras_vggface.vggface import VGGFace
import mtcnn
import numpy as np
import matplotlib as mpl
from keras.utils.data_utils import get_file
import dlib
import keras_vggface.utils
import PIL
import os
import os.path

In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from deepface import DeepFace
import pandas as pd

## Create Gradio Interface and Helper Function

In [5]:
models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcF/ace", 
  "Dlib", 
  "SFace",
]

In [6]:
#you will need to specify the path to your db
def db_find(path, frontalize, db="atfalmafkoda_unzip/database3", model=0, thresh=.2):
    """function to be passed into gradio that will check input image against the database and find matches
    args:
        path- path to image; can be numpy array as well
        align- bool; if True, frontalize image
        db- path to database
        model- 0-8; will load different models shown above (Dlib does not work)
        thresh- threshold to return images
    output:
        returns a dataframe of the matched images and a gallery of the actual images
    """
    if frontalize:
        print(type(path))
        path = align_face(path)[0]
        print(type(path))
    m = model
    dfs = DeepFace.find(img_path=path, db_path=db, model_name=models[m], detector_backend="mtcnn", enforce_detection=False)
    df = dfs[0].copy()
    df = df.drop(columns=['source_x', 'source_y', 'source_w', 'source_h'])
    df['id'] = df['identity'].str.strip("atfalmafkoda_unzip/database/person").str.split("/")
    df['id'] = df['id'].apply(lambda x: x[0])
    img_len = df.loc[df["VGG-Face_cosine"] < thresh].shape[0]
    imgs = df.head(img_len)['identity'].tolist()
    return df.loc[df["VGG-Face_cosine"] < thresh], imgs

In [7]:
# code to run gradio demo
demo = gr.Interface(fn=db_find, inputs=["image", gr.Checkbox(label="align")], outputs=["dataframe", "gallery"])
demo.launch(server_name='0.0.0.0', share=True)

IMPORTANT: You are using gradio version 3.9, however version 3.14.0 is available, please upgrade.
--------
Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://4e40cd6582c174a1.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x2ad6f232e5b0>,
 'http://localhost:7860/',
 'https://4e40cd6582c174a1.gradio.app')

There are  6707  representations found in  representations_vgg_face.pkl
1/1 [==============================] - 0s 125ms/step
find function lasts  25.706911325454712  seconds
